# Lựa chọn đặc trưng and LASSO

Trong notebook này, chúng ta sẽ sử dụng LASSO để lựa chọn đặc trưng, xây dựng solver đã triển khai trước cho LASSO (trong sklearn). Cụ thể, chúng ta sẽ:  
* Chạy LASSO với các L1 penalty khác nhau.
* Chọn L1 penalty tốt nhất sử dụng tập kiểm định.
* Chọn L1 penalty tốt nhất sử dụng tập kiểm định với ràng buộc bổ sung về kích thước tập con. 

Trong bài tập tiếp theo, chúng ta sẽ tạo LASSO solver sử dụng coordinate descent.

## Thư viện

In [1]:
import sklearn
import pandas as pd
import numpy as np

## Load tập dữ liệu doanh số bán nhà

Tập dữ liệu từ doanh số bán nhà quận King, Seatle, WA.

In [2]:
full_data = pd.read_csv("kc_house_data.csv", index_col=0)
full_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Tạo các đặc trung mới

Như ở lab 2 (*Lab-2.ipynb*), chúng ta sẽ xem xét các đặc trưng có các biến đổi đầu vào.

In [3]:
from math import log, sqrt
full_data['sqft_living_sqrt'] = full_data['sqft_living'].map(sqrt)
full_data['sqft_lot_sqrt'] = full_data['sqft_lot'].map(sqrt)
full_data['bedrooms_square'] = full_data['bedrooms'] ** 2

# Trong tập dữ liệu, 'floors' được xác định là type string, 
# nên chúng ta sẽ chuyển chúng thành float trước khi tạo đặc trưng mới. 
full_data['floors'] = full_data['floors'].astype(float) 
full_data['floors_square'] = full_data['floors'] ** 2

* Bình phương bedrooms sẽ tăng phân tách giữa ít phòng ngủ (chẳng hạn 1) và nhiều phòng ngủ (chẳng hạn 4) vì 1^2 = 1 còn 4^2 = 16. Do đó, biến này sẽ ảnh hưởng lớn tới các ngôi nhà có nhiều phòng ngủ.
* Mặt khác, căn bậc hai của sqft_living sẽ giảm phân tách giữa nhà lớn và nhà nhỏ. Chủ ngôi nhà cũng sẽ không vui hơn nếu nhà rộng gấp đôi.

# Tìm hiểu trọng số hồi quy với L1 penalty

Hãy khớp mô hình với tất cả đặc trưng hiện có cộng với các đặc trưng vừa tạo.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']
len(all_features)

17

Áp dụng L1 penalty cần thêm tham số (`alpha=l1_penalty`) cho mô hình  `Lasso` của Sklearn. (Các công cụ khác cũng phân tách các triển khai của LASSO). Khá giống Hồi quy Ridge/L2, các đặc trưng cũng cần được co giãn để đảm bảo đồng đều ở giữa.

In [5]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
l1_penalty=5e4
full_features = scaler.fit_transform(full_data[all_features].values)
full_labels = full_data['price'].values
model = Lasso(alpha=l1_penalty).fit(full_features, full_labels)

Tìm các đặc trưng có trọng số khác 0.

In [6]:
weights = model.coef_
weights

array([     0.        ,      0.        ,      0.        , 132571.09360631,
            0.        ,     -0.        ,     -0.        ,      0.        ,
            0.        ,  14623.33961421,  29004.06421249,      0.        ,
        90207.54789031,      0.        ,      0.        , -10722.34912003,
            0.        ])

In [7]:
features_ = []
weights_ = []
for i in range(len(all_features)):
    if weights[i] != 0:
        features_.append(all_features[i])
        weights_.append(weights[i])
features_,weights_

(['sqft_living', 'waterfront', 'view', 'grade', 'yr_built'],
 [132571.09360630723,
  14623.339614213912,
  29004.064212492078,
  90207.54789030869,
  -10722.34912002888])

Lưu ý rằng phần lớn trọng số được đặt thành 0. Vì vậy, bằng cách đặt L1 penalty đủ lớn, chúng ta có thể thực hiện lựa chọn tập con.

***QUIZ***:
Theo list các trọng số này, những đặc trưng nào đã được chọn?

# Lựa chọn L1 penalty

Để tìm một L1 penalty tốt, chúng ta sẽ khám phá nhiều giá trị sử dụng tập kiểm định. Hãy chia dữ liệu thành tập huấn luyện, tập kiểm định và tập kiểm tra:
* Chia dữ liệu bán hàng thành 2 tập: tập huấn luyện và tập kiểm tra (9/1)
* Chia tiếp tập huấn luyện thành 2 tập: tập huấn luyện và kiểm định (5/5)

Hãy dùng seed = 1 để có cùng kết quả!

In [8]:
from sklearn.model_selection import train_test_split
train_data_validation,test_data = train_test_split(full_data,test_size=0.1)
train_data,validation_data = train_test_split(train_data_validation,test_size= .5)

Tiếp theo, chúng ta sẽ viết một vòng lặp như sau: 
* Với `l1_penalty` trong phạm vi 21 bước giữa [1, 10^9] (sử dụng `np.logspace(0, 9, num=21)`.)
    * Khớp mô hình hồi quy với `l1_penalty` trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty` trong tham số.
    * Tính RSS trên dữ liệu KIỂM ĐỊNH (sử dụng `.predict()`) cho `l1_penalty`
* Báo lại `l1_penalty` nào cho RSS thấp nhất trong dữ liệu KIỂM ĐỊNH.

In [9]:
from sklearn.metrics import mean_squared_error

In [35]:
# Mức độ trợ giúp: Bình thường.
#List l1_penalty
full_l1_penalty = np.logspace(0,9,num = 21)
scaler = StandardScaler()
#Scaler train_data
full_features = scaler.fit_transform(train_data[all_features].values)
#Output train_data
labels_features = train_data['price'].values
#Scaler validation__data
full_test_features = scaler.fit_transform(validation_data[all_features].values)
#Output validation_data
labels_test_features = validation_data['price'].values
#Lưu các rss của từng l1_penalty
scores_rss_total = []
weights = []
for l1_penalty in full_l1_penalty:
    model = Lasso(alpha=l1_penalty).fit(full_features,labels_features)
    weights.append([l1_penalty,model.coef_])
    y_pr = model.predict(full_test_features)
#     scores_rss_total.append((np.sum((labels_test_features - y_pr)**2)))
    scores_rss_total.append(mean_squared_error(labels_test_features,y_pr))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14746664776971.562, tolerance: 137008208668.28061
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14755098819412.125, tolerance: 137008208668.28061
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14778776637397.812, tolerance: 137008208668.28061
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

***QUIZ:*** Giá trị tốt nhất cho `l1_penalty` là bao nhiêu?

In [36]:
scores_rss_total

[43418375780.266464,
 43414022566.73707,
 43401831607.1126,
 43368092891.65627,
 43277929719.21821,
 43062939591.882866,
 43091405474.138756,
 43163779167.837074,
 44990827361.32555,
 47682248400.4896,
 51926638010.497696,
 68053482849.71319,
 123786572954.2037,
 131890965680.98468,
 131890965680.98468,
 131890965680.98468,
 131890965680.98468,
 131890965680.98468,
 131890965680.98468,
 131890965680.98468,
 131890965680.98468]

In [37]:
weights

[[1.0,
  array([  18608.46071494,  -34841.73983882,   40461.31662553,
          277139.90242861, -443444.73378104,   26938.42754348,
          -46056.15501895,  -39597.82731676,   48052.37710637,
           47366.83495526,   32886.62493817,   18429.76101023,
          152589.88093391,  276841.19055042,  150326.5073571 ,
          -91144.63693162,    5749.05604438])],
 [2.8183829312644537,
  array([  18474.62548039,  -34717.84133315,   40455.39635834,
          277741.51002373, -443264.00270451,   26912.03496943,
          -46027.74122604,  -39463.73442899,   47925.51783197,
           47365.6112154 ,   32887.46822923,   18429.70917822,
          152589.28716547,  276144.13029242,  149958.31276474,
          -91140.00651586,    5746.48768689])],
 [7.943282347242816,
  array([  18097.42653989,  -34368.64789968,   40438.71077834,
          279437.07060109, -442754.63332359,   26837.6505892 ,
          -45947.66027708,  -39085.80932277,   47567.97981823,
           47362.16224787,   32889.

***QUIZ***
Với giá trị L1 penalty này, chúng ta có bao nhiêu trọng số khác 0?

In [38]:
for i in range(len(scores_rss_total)):
    if scores_rss_total[i] == min(scores_rss_total):
        print(scores_rss_total[i])
        print(full_l1_penalty[i])

43062939591.882866
177.82794100389228


In [39]:
l1_penalty_best = 177.82794100389228
model = Lasso(alpha=l1_penalty_best).fit(full_features,labels_features)
model.coef_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15482947217740.812, tolerance: 137008208668.28061
  model = cd_fast.enet_coordinate_descent(


array([   5593.70540624,  -22793.27786138,   39885.60254565,
        335642.99905437, -425869.61059723,   24371.89194287,
        -43293.06677663,  -26558.01657017,   35715.99628079,
         47247.83284834,   32968.63044879,   18424.72062713,
        152532.14015109,  209055.83689947,  114521.56744368,
        -90694.35394972,    5499.29712013])

# Limit the number of nonzero weights Giới hạn số trọng số khác 0

Sẽ ra sao nếu chúng ta muốn giới hạn, 5 đặc trưng chẳng hạn? Điều này quan trọng nếu chúng ta muốn suy ra "quy tắc ngón tay cái" --- mô hình có thể diễn giải chỉ với một vài đặc trưng.

Trong phần này, chúng ta sẽ triển khai quy trình đơn giản gồm 2 giai đoạn :
1. Thăm dò phạm vi lớn các giá trị `l1_penalty` để tìm vùng các giá trị `l1_penalty` hẹp hơn mà mô hình chắc chắn sẽ có số lượng trọng số khác 0 mong muốn.
2. Thăm dò tiếp vùng hẹp đã thấy để tìm gái trị tốt cho `l1_penalty` đạt được độ thưa thớt mong muốn. Ở đây chúng ta sử dụng tập kiểm định để chọn giá trị tốt nhất cho `l1_penalty`. 

In [40]:
max_nonzeros = 5

## Khám phá phạm vi giá trị lớn hơn để tìm phạm vi hẹp với độ thưa thớt mong muốn

Hãy xác định một loạt các `l1_penalty_values` có thể:

In [41]:
l1_penalty_values = np.logspace(3, 5, num=21)

Giờ hãy triển khi vòng lặp tìm kiếm trong không gian có các giá trị `l1_penalty` có thể:

* Với `l1_penalty` trong `np.logspace(3, 5, num=21)`:
    * Khớp mô hình hồi quy với `l1_penalty` đã biết trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty` trong tham số.
    * Trích xuất trọng số của mô hình và đếm số trọng số khác 0. Lưu con số này vào một list. 
        * Gợi ý: `model.coef_` cho các tham số/hệ số đã tìm thấy (intercept) ở dạng mảng numpy. Sau đó có thể dùng array\[condition\] cho list các giá trị truyền điều kiện, hoặc chỉ dùng `np.count_nonzero()` có sẵn.

In [43]:
#scaler features train
X = scaler.fit_transform(train_data[all_features].values)
y = train_data['price'].values
weights = []
for l1_penalty in l1_penalty_values:
    model = Lasso(alpha = l1_penalty).fit(X,y)
    weights.append([l1_penalty,model.coef_,np.count_nonzero(model.coef_)])
weights

[[1000.0,
  array([  -4284.88165957,  -13991.28869798,   36881.02834756,
          508486.92985361, -362888.90975284,   14297.81382322,
          -32414.64118385,      -0.        ,   11043.26177977,
           46853.79994364,   33538.04381831,   17176.47569839,
          151188.26648057,       0.        ,    3257.72200427,
          -90278.66843562,    4507.49909105]),
  15],
 [1258.9254117941675,
  array([ -10211.72229661,   -8608.76957293,   36253.64545958,
          486774.74662927, -340097.57197994,   11314.6850494 ,
          -29268.25718047,      -0.        ,   10997.3651937 ,
           46731.54491038,   33662.88701495,   16736.65409895,
          150698.67877036,       0.        ,    3035.94293234,
          -90255.17909063,    4213.46756548]),
  15],
 [1584.893192461114,
  array([ -17703.34935   ,   -1806.41227723,   35464.82551948,
          459322.60516757, -311287.02323329,    7565.99755511,
          -25312.54913227,       0.        ,   10938.45774715,
           46578.454

Trong phạm vi lớn này, chúng ta có thể tìm 2 đầu phạm vi hẹp mong muốn của `l1_penalty`. Ở một đầu, các giá trị `l1_penalty` có quá ít giá trị khác 0, còn đầu kia lại có quá nhiều giá trị khác 0.

Hãy tìm:
* `l1_penalty` nhỏ nhất có các số khác 0 bằng `max_nonzeros` (nếu chọn penalty nhỏ hơn giá trị này chắc chắn sẽ có rất nhiều trọng số khác 0).
    * Lưu giá trị này trong biến `l1_penalty_min` (sẽ sử dụng nó sau).
* `l1_penalty` lớn nhất có các số khác 0 bằng `max_nonzeros` (nếu chọn penalty lớn hơn giá trị này chắc chắn sẽ có rất ít trọng số khác 0).
    * Lưu giá trị này trong biến `l1_penalty_max` (sẽ sử dụng nó sau).


*Gợi ý: có nhiều cách để thực hiện, chẳng hạn:*
* Lập trình trong vòng lặp trên.
* Tạo một list với số lượng khác 0 cho từng giá trị `l1_penalty` và kiểm tra nó để tìm ranh giới thích hợp.

In [45]:
l1_penalty_list = [i for i,j,k in weights if k == max_nonzeros]

In [46]:
l1_penalty_min = min(l1_penalty_list)
l1_penalty_max = max(l1_penalty_list)

***QUIZ.*** Chúng ta tìm thấy các giá trị nào lần lượt cho `l1_penalty_min` và `l1_penalty_max`? 

In [47]:
l1_penalty_min,l1_penalty_max

(19952.62314968879, 50118.72336272725)

## Khám phá phạm vi nhỏ các giá trị để tìm giải pháp với đúng số lượng các số khác 0 có RSS trong tập kiểm định nhỏ nhất

Chúng ta sẽ khám phá vùng hẹp các giá trị `l1_penalty` đã tìm thấy:

In [49]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* Với `l1_penalty` trong `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Khớp mô hình hồi quy với `l1_penalty` đã biết trong dữ liệu HUẤN LUYỆN. Chỉ định `alpha=l1_penalty`.
    * Đo lường RSS của mô hình đã tìm hiểu trong tập KIỂM ĐỊNH.

Tìm mô hình có RSS nhỏ nhất trong tập KIỂM ĐỊNH và độ thưa thớt *bằng*  `max_nonzeros`.

In [54]:
#scaler features train
X = scaler.fit_transform(train_data[all_features].values)
y = train_data['price'].values
#Scaler feature validation
X_test = scaler.fit_transform(validation_data[all_features].values)
y_test = validation_data['price'].values
total = []
scores_rss = []
for l1_penalty in l1_penalty_values:
    model = Lasso(alpha = l1_penalty).fit(X,y)
    
    y_predict = model.predict(X_test)
    scores_rss.append(np.sum((y_test - y_predict)**2))
    
    total.append([l1_penalty,model.coef_,np.count_nonzero(model.coef_),np.sum((y_test - y_predict)**2)])

***QUIZ***
1. Giá trị của `l1_penalty` trong phạm vi hẹp hơn có RSS thấp nhất trong tập KIỂM ĐỊNH và độ thưa *bằng* `max_nonzeros` là?
2. Các đặc trung nào trong mô hình này có các hệ số khác 0?

In [55]:
total

[[19952.62314968879,
  array([    -0.        ,     -0.        ,      0.        , 153773.97046634,
              0.        ,     -0.        ,     -0.        ,      0.        ,
              0.        ,  35013.72454855,  36063.14124195,      0.        ,
         130186.84318353,      0.        ,      0.        , -59656.57986769,
              0.        ]),
  5,
  481437471310811.2],
 [21540.31263458555,
  array([    -0.        ,     -0.        ,      0.        , 153180.56933694,
              0.        ,     -0.        ,     -0.        ,      0.        ,
              0.        ,  33799.36721432,  35873.32504476,      0.        ,
         128011.4604752 ,      0.        ,      0.        , -56967.83169816,
              0.        ]),
  5,
  483977800369247.7],
 [23128.00211948231,
  array([    -0.        ,     -0.        ,      0.        , 152578.0579042 ,
              0.        ,     -0.        ,     -0.        ,      0.        ,
              0.        ,  32585.51733937,  35682.6092065

In [61]:
l1_penalty = [i  for i,j,k,l in total if l == min(scores_rss)]
l1_penalty

[19952.62314968879]

In [66]:
weights = [j for i,j,k,l in total if l == min(scores_rss)]
weights = [j for i in weights for j in i]
weights

[-0.0,
 -0.0,
 0.0,
 153773.9704663409,
 0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 35013.724548548584,
 36063.14124195357,
 0.0,
 130186.84318353128,
 0.0,
 0.0,
 -59656.57986768807,
 0.0]

In [67]:
features_ = []
weights_ = []
for i in range(len(all_features)):
    if weights[i] != 0:
        features_.append(all_features[i])
        weights_.append(weights[i])
features_,weights_

(['sqft_living', 'waterfront', 'view', 'grade', 'yr_built'],
 [153773.9704663409,
  35013.724548548584,
  36063.14124195357,
  130186.84318353128,
  -59656.57986768807])